In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [44]:
%%time
df = pd.read_excel('card transactions.xlsx')

Wall time: 15.9 s


# 1. Drop Outliners and Keep Only P

In [45]:
df.columns

Index(['Recnum', 'Cardnum', 'Date', 'Merchnum', 'Merch description',
       'Merch state', 'Merch zip', 'Transtype', 'Amount', 'Fraud'],
      dtype='object')

In [46]:
#Drop one large transaction
#df['Amount'].max()
#df[df['Amount']== 3102045.53]
df = df.drop(52715)

In [47]:
#Keep only type P
df = df[df['Transtype']=='P']

# 2.Filling in missing values

In [48]:
#filling in merch zip by merch state and merchnum

In [49]:
zip_ref_1 = df.groupby(['Merch state','Merchnum'])['Merch zip'].apply(lambda x: x.mode()).reset_index()
zip_ref_1 = zip_ref_1[zip_ref_1['level_2'] == 0].iloc[:,[0,1,3]]
zip_ref_1 = zip_ref_1.rename(columns = {'Merch zip':'zip_ref_1'})

In [50]:
df = df.merge(zip_ref_1,on = ['Merch state','Merchnum'], how = 'left')

In [51]:
zip_ref_2 = df.groupby('Merch state')['Merch zip'].apply(lambda x: x.mode()).reset_index()
zip_ref_2 = zip_ref_2[zip_ref_2['level_1'] == 0].iloc[:,[0,2]]
zip_ref_2 = zip_ref_2.rename(columns = {'Merch zip':'zip_ref_2'})

In [52]:
df = df.merge(zip_ref_2,on = 'Merch state', how = 'left')

In [53]:
zip_ref_3 = df.groupby('Merchnum')['Merch zip'].apply(lambda x: x.mode()).reset_index()
zip_ref_3 = zip_ref_3[zip_ref_3['level_1'] == 0].iloc[:,[0,2]]
zip_ref_3 = zip_ref_3.rename(columns = {'Merch zip':'zip_ref_3'})

In [54]:
df = df.merge(zip_ref_3,on = 'Merchnum', how = 'left')

In [55]:
df['Merch zip'] = df['Merch zip'].fillna(df['zip_ref_1']).fillna(df['zip_ref_2']).fillna(df['zip_ref_3'])
sum(pd.isnull(df['Merch zip']))

1096

In [56]:
#drop if the value is still na
df = df.dropna(subset=['Merch zip'])
sum(pd.isnull(df['Merch zip']))

0

In [57]:
#filling in merch state by merch zip and merchnum
state_ref_1 = df.groupby(['Merch zip','Merchnum'])['Merch state'].apply(lambda x: x.mode()).reset_index()
state_ref_1 = state_ref_1[state_ref_1['level_2'] == 0].iloc[:,[0,1,3]]
state_ref_1 = state_ref_1.rename(columns = {'Merch state':'state_ref_1'})

In [58]:
df = df.merge(state_ref_1,on = ['Merch zip','Merchnum'], how = 'left')

In [59]:
state_ref_2 = df.groupby('Merch zip')['Merch state'].apply(lambda x: x.mode()).reset_index()
state_ref_2 = state_ref_2[state_ref_2['level_1'] == 0].iloc[:,[0,2]]
state_ref_2 = state_ref_2.rename(columns = {'Merch state':'state_ref_2'})

In [60]:
df = df.merge(state_ref_2,on = 'Merch zip', how = 'left')

In [61]:
state_ref_3 = df.groupby('Merchnum')['Merch state'].apply(lambda x: x.mode()).reset_index()
state_ref_3 = state_ref_3[state_ref_3['level_1'] == 0].iloc[:,[0,2]]
state_ref_3 = state_ref_3.rename(columns = {'Merch state':'state_ref_3'})

In [62]:
df = df.merge(state_ref_3,on = 'Merchnum', how = 'left')

In [63]:
df['Merch state'] = df['Merch state'].fillna(df['state_ref_1']).fillna(df['state_ref_2']).fillna(df['state_ref_3'])
sum(pd.isnull(df['Merch state']))

67

In [64]:
#drop if the value is still na
df = df.dropna(subset=['Merch state'])
sum(pd.isnull(df['Merch state']))

0

In [65]:
#filling in merchnum by merch zip and merch state
num_ref_1 = df.groupby(['Merch zip','Merch state'])['Merchnum'].apply(lambda x: x.mode()).reset_index()
num_ref_1 = num_ref_1[num_ref_1['level_2'] == 0].iloc[:,[0,1,3]]
num_ref_1 = num_ref_1.rename(columns = {'Merchnum':'num_ref_1'})

In [66]:
df = df.merge(num_ref_1,on = ['Merch zip','Merch state'], how = 'left')

In [67]:
num_ref_2 = df.groupby('Merch zip')['Merchnum'].apply(lambda x: x.mode()).reset_index()
num_ref_2 = num_ref_2[num_ref_2['level_1'] == 0].iloc[:,[0,2]]
num_ref_2 = num_ref_2.rename(columns = {'Merchnum':'num_ref_2'})

In [68]:
df = df.merge(num_ref_2,on = 'Merch zip', how = 'left')

In [69]:
num_ref_3 = df.groupby('Merch state')['Merchnum'].apply(lambda x: x.mode()).reset_index()
num_ref_3 = num_ref_3[num_ref_3['level_1'] == 0].iloc[:,[0,2]]
num_ref_3 = num_ref_3.rename(columns = {'Merchnum':'num_ref_3'})

In [70]:
df = df.merge(num_ref_3,on = 'Merch state', how = 'left')

In [71]:
df['Merchnum'] = df['Merchnum'].fillna(df['num_ref_1']).fillna(df['num_ref_2']).fillna(df['num_ref_3'])
sum(pd.isnull(df['Merchnum']))

0

In [72]:
# drop all reference columns
df = df.drop(['state_ref_1', 'state_ref_2', 'state_ref_3','zip_ref_1', 'zip_ref_2', 'zip_ref_3','num_ref_1', 'num_ref_2', 'num_ref_3'], axis=1)

In [73]:
df.to_csv('fdata.csv')

In [74]:
df.shape

(95234, 10)

# 3. Variable Creation

## 3.1. Amount and frequency variables

In [32]:
#days = [1,3,7,10,14,30]
aggs = {'Amount':['mean', 'median', 'max', 'sum', 'count']}

In [33]:
df.set_index('Date', inplace = True)

In [34]:
%%time
card = []
days = [1,3,7,14,30, 90]
for day in days:
    a = df.groupby(by = ['Cardnum']).Amount.rolling(str(day)+'d').agg(aggs).Amount.reset_index()
    a['order_card'] = a.groupby(['Cardnum', 'Date']).cumcount() + 1
    card.append(a)

C:\Users\clair\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\window.py:1561: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(Rolling, self).aggregate(arg, *args, **kwargs)


Exception: Column(s) Amount already selected

In [35]:
%%time

merch = []
for day in days:
    a = df[['Merchnum', 'Amount']].groupby(by = 'Merchnum')['Amount'].rolling(str(day)+'d').agg(aggs).Amount.reset_index()
    a['order_merch'] = a.groupby(['Date', 'Merchnum']).cumcount() + 1
    merch.append(a)

C:\Users\clair\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\window.py:1561: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(Rolling, self).aggregate(arg, *args, **kwargs)


Exception: Column(s) Amount already selected

In [36]:
%%time
state = []
for day in days:
    a = df[['Cardnum', 'Merch state', 'Amount']].groupby(by = ['Cardnum','Merch state']).Amount.rolling(str(day)+'d')\
    .agg(aggs).Amount.reset_index()
    a['order_state'] = a.groupby(['Date', 'Merch state', 'Cardnum']).cumcount() + 1
    state.append(a)

C:\Users\clair\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\window.py:1561: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(Rolling, self).aggregate(arg, *args, **kwargs)


Exception: Column(s) Amount already selected

In [37]:
%%time
zip = []
for day in days:
    a = df.groupby(by = ['Cardnum' ,'Merch zip']).Amount.rolling(str(day)+'d').agg(aggs).Amount.reset_index()
    a['order_zip'] = a.groupby(['Date', 'Merch zip', 'Cardnum']).cumcount() + 1
    zip.append(a)

C:\Users\clair\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\window.py:1561: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(Rolling, self).aggregate(arg, *args, **kwargs)


Exception: Column(s) Amount already selected

In [38]:
%%time
cardmerch = []
for day in days:
    a = df.groupby(by = ['Cardnum','Merchnum']).Amount.rolling(str(day)+'d').agg(aggs).Amount.reset_index()
    a['order_mc'] = a.groupby(['Date', 'Merchnum', 'Cardnum']).cumcount() + 1
    cardmerch.append(a)

C:\Users\clair\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\window.py:1561: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(Rolling, self).aggregate(arg, *args, **kwargs)


Exception: Column(s) Amount already selected

In [39]:
df['order_merch'] = df.groupby(by = ['Date', 'Merchnum']).cumcount() + 1
df['order_card'] = df.groupby(by = ['Date', 'Cardnum']).cumcount() + 1
df['order_mc'] = df.groupby(by = ['Date', 'Cardnum', 'Merchnum']).cumcount() + 1
df['order_zip'] = df.groupby(by = ['Date', 'Cardnum', 'Merch zip']).cumcount() + 1
df['order_state'] = df.groupby(by = ['Date', 'Cardnum', 'Merch state']).cumcount() + 1

In [40]:
temp = card[0]
i = 0
for c in card[1:]:
    y = days[i]
    x = days[1+i]
    i+=1
    temp = pd.merge(left = temp, right = c, how = 'inner', on = ['Date', 'order_card', 'Cardnum'], suffixes = \
                    ('card_day'+str(y), 'card_day'+str(x)))
temp.head()

IndexError: list index out of range

In [ ]:
#

In [ ]:
merchs = merch[0]
i = 0
for c in merch[1:]:
    y = days[i]
    x = days[1+i]
    i+=1
    merchs = pd.merge(left = merchs, right = c, how = 'inner', on = ['Date', 'order_merch', 'Merchnum'], suffixes = \
                    ('merch_day'+str(y), 'merch_day'+str(x)))
merchs.head()

In [ ]:
states = state[0]
i = 0
for c in state[1:]:
    y = days[i]
    x = days[1+i]
    i+=1
    states = pd.merge(left = states, right = c, how = 'inner', on = ['Date', 'order_state', 'Cardnum', 'Merch state'], suffixes = \
                    ('state_day'+str(y), 'state_day'+str(x)))
states.head()

In [ ]:
zips = zip[0]
i = 0
for c in zip[1:]:
    y = days[i]
    x = days[1+i]
    i+=1
    zips = pd.merge(left = zips, right = c, how = 'inner', on = ['Date', 'order_zip', 'Cardnum', 'Merch zip'], suffixes = \
                    ('zip_day'+str(y), 'zip_day'+str(x)))
zips.head()

In [ ]:
mc = cardmerch[0]
i = 0
for c in cardmerch[1:]:
    y = days[i]
    x = days[1+i]
    i+=1
    mc = pd.merge(left = mc, right = c, how = 'inner', on = ['Date', 'order_mc', 'Cardnum', 'Merchnum'], suffixes = \
                    ('mc_day'+str(y), 'mc_day'+str(x)))
mc.head()

In [ ]:
for i in df.columns[15:].unique():
    del df[i]

# merging tables

In [ ]:
df = df.merge(temp, how = 'left', on = ['Date', 'Cardnum', 'order_card'], suffixes = ('', '_card'))

In [ ]:
df = df.merge(merchs, how = 'left', on = ['Date', 'Merchnum', 'order_merch'], suffixes = ('', '_merch'))

In [ ]:
df = df.merge(mc, how = 'left', on = ['Date', 'Cardnum', 'Merchnum', 'order_mc'], suffixes = ('', '_mc'))

In [ ]:
df = df.merge(zips, how = 'left', on = ['Date', 'Cardnum', 'Merch zip', 'order_zip'], suffixes = ('', '_zip'))

In [ ]:
df = df.merge(states, how = 'left', on = ['Date', 'Cardnum', 'Merch state', 'order_state'], suffixes = ('', '_state'))

In [ ]:
#DAY 0

In [ ]:
df.columns

In [ ]:
%%time
z = df[df.columns[:10]].groupby(['Cardnum', 'Date']).Amount.rolling(100, min_periods = 0).agg(aggs).Amount.reset_index()
z['order_card'] = z.groupby(['Cardnum', 'Date']).cumcount()+1

In [ ]:
z2 = df[df.columns[:10]].groupby(['Merchnum', 'Date']).Amount.rolling(100, min_periods = 0).agg(aggs).Amount.reset_index()

z2['order_merch'] = z2.groupby(['Merchnum', 'Date']).cumcount()+1

In [ ]:
z3 = df[df.columns[:10]].groupby(['Merchnum','Cardnum', 'Date']).Amount.rolling(100,  min_periods = 0).agg(aggs)\
.Amount.reset_index()

z3['order_mc'] = z3.groupby(['Merchnum','Cardnum', 'Date']).cumcount()+1

In [ ]:
z4 = df[df.columns[:10]].groupby(['Cardnum','Merch state', 'Date']).Amount.rolling(100,  min_periods = 0).agg(aggs).\
Amount.reset_index()

z4['order_state'] = z4.groupby(['Cardnum','Merch state', 'Date']).cumcount()+1

In [ ]:
%%time
z5 = df[df.columns[:10]].groupby(['Cardnum','Merch zip', 'Date']).Amount.rolling(100, min_periods = 0).\
agg(aggs).Amount.reset_index()

z5['order_zip'] = z5.groupby(['Cardnum','Merch zip', 'Date']).cumcount()+1

In [ ]:
z['level_2'] +=1

In [ ]:
z2['level_2'] +=1

In [ ]:
z3['level_3'] +=1
z4['level_3'] +=1
z5['level_3'] +=1

In [ ]:
z['Recnum'] = z['level_2']
z2['Recnum'] = z2['level_2']
z3['Recnum'] = z3['level_3']
z4['Recnum'] = z4['level_3']
z5['Recnum'] = z5['level_3']


del z['level_2']
del z2['level_2']
del z3['level_3']
del z4['level_3']
del z5['level_3']


In [ ]:
del z['Recnum']
del z2['Recnum']
del z3['Recnum']
del z4['Recnum']
del z5['Recnum']

In [ ]:
df = df.merge(z, how = 'left', on = ['Cardnum', 'Date', 'order_card'],   suffixes = ('','_card_day0'))


In [ ]:
df = df.merge(z2, how = 'left', on = ['Merchnum', 'Date', 'order_merch'], suffixes = ('','_merch_day0'))

In [ ]:
df = df.merge(z3, how = 'left', on = ['Merchnum', 'Date','Cardnum', 'order_mc'], suffixes = ('','_mc_day0'))

In [ ]:
df = df.merge(z4, how = 'left', on = ['Merch state', 'Date','Cardnum', 'order_state'\
                                      ], suffixes = ('','_state_day0'))

In [ ]:
df = df.merge(z5, how = 'left', on = ['Merch zip', 'Date','Cardnum', \
                                      'order_zip'], suffixes = ('','_zip_day0'))

In [ ]:
pd.Series(df.columns).value_counts()

In [ ]:
df.columns[170:190]

In [ ]:
c = df.columns[15:]

In [ ]:
j = 0
i = 0
for col in c:
    j = j%5
    if j==4:
        j+=1
        continue
    j+=1
    i+=1
    df['Amount/'+str(col)+'_'+str(i)] = df['Amount']/df[col]

# Days since variable

In [ ]:
def second_last(x):
    a = sorted(x, reverse = True)
    if len(a) ==1:
        return 0
    else:
        return a[1]

In [ ]:
a = df.set_index('Date').groupby([ 'Cardnum']).Recnum.rolling('365d').apply(lambda x: second_last(x)).reset_index()
a['order_card'] = a.groupby(['Cardnum']).cumcount()+1
a= a.merge(df[['Recnum', 'Date']], how = 'inner', suffixes = ('','_previous_card'), on = 'Recnum')

In [ ]:
b = df.set_index('Date').groupby([ 'Merchnum']).Recnum.rolling('365d').apply(lambda x: second_last(x)).reset_index()
b['order_merch'] = b.groupby(['Merchnum']).cumcount()+1
b= b.merge(df[['Recnum', 'Date']], how = 'inner', suffixes = ('','_previous_merch'), on = 'Recnum')

In [ ]:
c = df.set_index('Date').groupby([ 'Cardnum', 'Merch zip']).Recnum.\
rolling('365d').apply(lambda x: second_last(x)).reset_index()

c['order_zip'] = c.groupby(['Cardnum', 'Merch zip']).cumcount()+1
c = c.merge(df[['Recnum', 'Date']], how = 'inner', suffixes = ('','_previous_zip'), on = 'Recnum')

In [ ]:
d = df.set_index('Date').groupby([ 'Cardnum', 'Merch state']).Recnum.\
rolling('365d').apply(lambda x: second_last(x)).reset_index()

d['order_state'] = d.groupby(['Cardnum', 'Merch state']).cumcount()+1

d = d.merge(df[['Recnum', 'Date']], how = 'inner', suffixes = ('','_previous_state'), on = 'Recnum')

In [ ]:
e = df.set_index('Date').groupby([ 'Cardnum', 'Merchnum']).Recnum.\
rolling('365d').apply(lambda x: second_last(x)).reset_index()

e['order_mc'] = e.groupby(['Cardnum', 'Merchnum']).cumcount()+1

e = e.merge(df[['Recnum', 'Date']], how = 'inner', suffixes = ('','_previous_mc'), on = 'Recnum')

In [ ]:
del a['Recnum']
del b['Recnum']
del e['Recnum']
del d['Recnum']
del c['Recnum']


In [ ]:
#df = 

df = df.merge(e, on = ['Cardnum', 'Merchnum', 'Date', 'order_mc'],how = 'left', suffixes = ('',''))

df = df.merge(a, on = ['Cardnum', 'Date', 'order_card'], suffixes = ('',''), how = 'left')

df = df.merge(c, on = ['Cardnum', 'Merch zip', 'Date', 'order_zip'], suffixes = ('',''), how = 'left')

df = df.merge(b, on = [ 'Merchnum', 'Date', 'order_merch'], suffixes = ('',''), how = 'left')

In [ ]:
df = df.merge(d, on = ['Cardnum', 'Merch state', 'Date', 'order_state'], suffixes = ('',''), how = 'left')

In [ ]:
#start with date operations

In [ ]:
df['Date']= df['Date'].apply(lambda x: x.date())


In [ ]:
x = df['Date']



In [ ]:
df[df.columns[330:]]

In [ ]:
for i in df.columns[330:]:
    df['Date'] - df[i]

# KS test

In [ ]:
from scipy.stats import ks_2samp

In [ ]:
d = {}
b = df[df['Fraud']==1]
a = df[df['Fraud']==0]
for i in df.columns[15:]:
    x = a[i]
    y = b[i] 

    d[i] = ks_2samp(x, y)[0]

In [ ]:
l = []
[l.append([key, float(val)]) for key, val in d.items()]

ks = pd.DataFrame(l, columns = ['Column', 'KS Score'])

In [ ]:
ks = ks.sort_values(by = 'KS Score', ascending=False)

In [ ]:
#number of ks above 50%
sum(ks['KS Score']>0.5)

In [ ]:
ks.tail()

In [ ]:
ks.head()

In [ ]:
df.columns

In [ ]:
df.to_csv('transactions.csv', encoding='utf-8', index=False)

In [ ]:
df.shape

In [ ]:
round(df, 2)